In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm
from kiwipiepy import Kiwi

tqdm.pandas()

In [2]:
df = pd.read_csv('df_all_small1.csv',encoding='utf-8-sig')

In [3]:
def cleansing(body, mode=None):
    if type(body) == np.ndarray:
        body = np.round(body.tolist(), 5)
        body = body.tolist()
        return body
    else:
        body = re.sub('<YNAOBJECT.*?/YNAOBJECT>', '', body, 0, re.I|re.S) # YNAOBJECT 태그 제거
        body = re.sub('<table.*?/table>', '', body, 0, re.I|re.S) # table 태그 제거
        
        if mode == 'meta':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[0]:]
            except AttributeError:
                pass
            return body
        
        if mode == 'sum':
            try:
                body = body[re.search("\(.*?연합뉴스\).*?=", body).span()[1]:]
            except AttributeError:
                pass
            
        
        body = re.sub(r'\([^)]*\)', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\[.*\]|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub(r'\<.*\>|\s-\s.*', '', body, 0, re.I|re.S) # 괄호안 내용 제거
        body = re.sub('·', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('-', ' ', body, 0, re.I|re.S) # 변환
        body = re.sub('\(.*?연합뉴스\).*?=', '', body, 0, re.I|re.S) # 기자명 및 기자소속 제거
        body = re.sub('\(서울.*?연합뉴스\)', '', body, 0, re.I|re.S) # 기자소속 제거
        body = re.sub('yna.co.kr', '', body, 0, re.I|re.S) # 연합 url 제거
        body = re.sub('([a-zA-Z0-9\_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+)', '', body, 0, re.I|re.S) # E-mail 제거
        body = re.sub('(http|ftp|https)://(?:[-\w.]|(?:%[\da-fA-F]{2}))+', '', body, 0, re.I|re.S) # URL 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+억원)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+억[0-9-]+천만달러)', '', body, 0, re.I|re.S) # 억천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+만원)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+천[0-9-]+달러)', '', body, 0, re.I|re.S) # 천만 제거
#         body = re.sub('([0-9\_.+-]+만원)', '', body, 0, re.I|re.S) # 만 제거
#         body = re.sub('([0-9\_.+-]+만[0-9-]천[0-9-]+원)', '', body, 0, re.I|re.S) # 만 제거
        body = re.sub('전문보기.*?\n', '', body, 0, re.I|re.S) # 전문보기 제거
        body = re.sub('&lt;.*?&gt;&gt;', '', body, 0, re.I|re.S) # 기타 안내문 제거
        body = re.sub('&lt;.*?&gt;', '', body, 0, re.I|re.S) # '&gt;'가 1개인 안내문 제거
        body = re.sub('&apos;', "'", body, 0, re.I|re.S) # &apos; ' 변환
        body = re.sub('&amp;', "&", body, 0, re.I|re.S) # &amp;; ' 변환        
        body = re.sub('-.*?_.*?\(끝\)', '', body, 0, re.I|re.S) # 9001887 속성에만 존재하는 불용어 제거
        body = re.sub('[#@*※|》■▲▼◀▶◇☎]-<>=+·↑', '', body, 0, re.I|re.S) # 특수문자 제거
        body = re.sub('…', '', body, 0, re.I|re.S) # … 제거
        body = re.sub('\(끝\)', '', body, 0, re.I|re.S) # 끝 태그 제거
        body = re.sub('[""]', '', body, 0, re.I|re.S) # "" 제거
        body = re.sub(r'[^ A-Za-z가-힣+]',' ' , body, 0, re.I|re.S) # 한글, 알파벳을 제외한 나머지 제거


        paragraphs = body.split('\r\n') # 단락 분리
        if ' 기자' in paragraphs[-1]: del paragraphs[-1] # 끝 단락 기자명 제거
        body = []
        for paragraph in paragraphs:
            paragraph = re.sub('\[.+?\]', '', paragraph, 0, re.I|re.S)
            paragraph = ' '.join(paragraph.split()) # 문자열 중간 다중 공백 제거
            if len(paragraph) > 0:
                body.append(paragraph)
        
        if mode == 'quot':
            return body
        else:
            return ' '.join(body)

In [4]:
df.columns

Index(['YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body', 'NamedEntity'], dtype='object')

In [5]:
df = df[['YNewsML.NewsContent.Title','clean','YNewsML.Header.SendDate']]

KeyError: "['clean', 'YNewsML.Header.SendDate'] not in index"

In [6]:
df1 = df.drop_duplicates()

In [8]:
df2 = df1.dropna(subset=['NamedEntity']).reset_index().drop(columns='index',axis=1)

In [9]:
clean = df2.clean

AttributeError: 'DataFrame' object has no attribute 'clean'

In [9]:
a = []
for i in range(len(clean)):
        if len(str(clean[i])) <= 15:
                a.append([i,clean[i]])

In [10]:
b = pd.DataFrame(a)

In [11]:
b1 = b[0].tolist()

In [12]:
df3 = df2.drop(index = b1)

In [13]:
df3 = df3.reset_index().drop(columns='index',axis=1)

In [14]:
df3['YNewsML.NewsContent.Title']

0                 인니 대표 스타트업 고젝, 토코피디아와 합병 논의
1                 중앙아프리카공화국 대선 결과 투아데레 대통령 재선
2         당국, 동부구치소 사망자 늑장통보 논란에 "화장 전에 통보했다"
3                  삼성전자 사흘째 최고가 경신…시총 500조 시대
4              지아이그룹-세브란스병원, 혁신 신약 개발 공동연구 협약
                         ...                 
690977         [酒먹방] 다섯 차례 빚어 천천히 숙성한 '느림의 술'
690978      [imazine] 쉼이 있는 여행 ③ 전등사 한옥 템플스테이
690979            [일지] 아프간 전쟁 발발부터 미군 철수 발표까지
690980           러 극동 연해주서 돼지열병 확산세…올해 59건 발생
690981                                  ㅇㅇㅇㅇㅇ
Name: YNewsML.NewsContent.Title, Length: 690982, dtype: object

In [15]:
df3

,YNewsML.NewsContent.Title,clean,YNewsML.Header.SendDate
0,"인니 대표 스타트업 고젝, 토코피디아와 합병 논의","인니 대표 스타트업 고젝, 토코피디아와 합병 논의 인도네시아 최대 스타트업으로 평가...",20210105.0
1,중앙아프리카공화국 대선 결과 투아데레 대통령 재선,중앙아프리카공화국 대선 결과 투아데레 대통령 재선 폭력 사태 속 치러진 선거서 과반...,20210105.0
2,"당국, 동부구치소 사망자 늑장통보 논란에 ""화장 전에 통보했다""","당국, 동부구치소 사망자 늑장통보 논란에 ""화장 전에 통보했다"" 서울동부구치소에서 ...",20210105.0
3,삼성전자 사흘째 최고가 경신…시총 500조 시대,삼성전자 사흘째 최고가 경신…시총 500조 시대 삼성전자가 5일 사흘째 최고가를 경...,20210105.0
4,"지아이그룹-세브란스병원, 혁신 신약 개발 공동연구 협약","지아이그룹세브란스병원, 혁신 신약 개발 공동연구 협약 신약 개발기업 지아이이노베이션...",20210105.0
...,...,...,...
690977,[酒먹방] 다섯 차례 빚어 천천히 숙성한 '느림의 술',다섯 차례 빚어 천천히 숙성한 '느림의 술' 평택 좋은술의 오양주 '천비향' 전통주...,20201023.0
690978,[imazine] 쉼이 있는 여행 ③ 전등사 한옥 템플스테이,쉼이 있는 여행 ③ 전등사 한옥 템플스테이 환란 벗어나 피안 '강 저쪽 둔덕'이라는...,20201022.0
690979,[일지] 아프간 전쟁 발발부터 미군 철수 발표까지,아프간 전쟁 발발부터 미군 철수 발표까지 조 바이든 미국 대통령이 14일(현지시간)...,20210415.0
690980,러 극동 연해주서 돼지열병 확산세…올해 59건 발생,"러 극동 연해주서 돼지열병 확산세…올해 59건 발생 당국 ""야생 멧돼지 활동 증가가...",20201117.0


In [23]:
result2 = df3['YNewsML.NewsContent.Title'].progress_apply(cleansing)

100%|██████████| 690982/690982 [06:35<00:00, 1746.84it/s]


In [24]:
result = df3.clean.progress_apply(cleansing)

100%|██████████| 690982/690982 [3:31:38<00:00, 54.42it/s]   


In [25]:
result = pd.DataFrame(result)

In [26]:
result2 = pd.DataFrame(result2)

In [27]:
result3 = pd.concat([result,result2,df3['YNewsML.Header.SendDate']], axis=1)

In [28]:
result3

,clean,YNewsML.NewsContent.Title,YNewsML.Header.SendDate
0,인니대표스타트업고젝토코피디아와합병논의인도네시아최대스타트업으로평가받는고젝과토코피디아가...,인니대표스타트업고젝토코피디아와합병논의,20210105.0
1,중앙아프리카 공화국 대선 결과 투 아 데 레 대통령 재선 폭력 사태 속 치러진 선거...,중앙아프리카 공화국 대선 결과 투 아 데 레 대통령 재선,20210105.0
2,당국 동부 구치소 사망자 늑장 통보 논란에 화장 전에 통보했다 서울 동부 구치소에서...,당국 동부 구치소 사망자 늑장 통보 논란에 화장 전에 통보했다,20210105.0
3,삼성전자 사흘째 최고가 경신 시 총 조 시대 삼성전자가 일 사흘째 최고가를 경신했다...,삼성전자 사흘째 최고가 경신 시 총 조 시대,20210105.0
4,지아이그룹 세브란스병원 혁신 신약 개발 공동 연구 협약 신약 개발 기업지 아이 이노...,지아이그룹 세브란스병원 혁신 신약 개발 공동 연구 협약,20210105.0
...,...,...,...
690977,다섯 차례 빚어 천천히 숙성한 느림의 술 평택 좋은 술의 오 양주 천 비 향 전통주...,다섯 차례 빚어 천천히 숙성한 느림의 술,20201023.0
690978,쉼이 있는 여행 전등사 한옥 템플스테이 환란 벗어나 피안 강 저 쪽 둔덕이라는 뜻을...,쉼이 있는 여행 전등사 한옥 템플스테이,20201022.0
690979,아프간 전쟁 발발부터 미군 철수 발표까지 조 바이든 미국 대통령이 일 아프가니스탄 ...,아프간 전쟁 발발부터 미군 철수 발표까지,20210415.0
690980,러 극동 연해주서 돼지 열병 확산 세 올해 건 발생 당국 야생 멧돼지 활동 증가가 ...,러 극동 연해주서 돼지 열병 확산 세 올해 건 발생,20201117.0


In [29]:
result4 = result3.clean + '. ' +result3['YNewsML.NewsContent.Title']

In [30]:
result5 = pd.concat([result3,result4],axis=1)

In [31]:
result5 = result5.rename(columns={'YNewsML.NewsContent.Title':'title',0:'TB','YNewsML.Header.SendDate':'Date'})

In [32]:
result5['Date'] = pd.to_datetime(result5['Date'].astype(int).astype(str))

In [33]:
result5.isnull().sum()

clean    0
title    0
Date     0
TB       0
dtype: int64

In [34]:
result5.to_csv('cleansing_space.csv',encoding='utf-8-sig',index=False)

In [35]:
df1 = df1.reset_index().drop(columns='index',axis=1)

In [36]:
df1

,YNewsML.NewsContent.Title,clean,YNewsML.Header.SendDate
0,"인니 대표 스타트업 고젝, 토코피디아와 합병 논의","인니 대표 스타트업 고젝, 토코피디아와 합병 논의 인도네시아 최대 스타트업으로 평가...",20210105.0
1,중앙아프리카공화국 대선 결과 투아데레 대통령 재선,중앙아프리카공화국 대선 결과 투아데레 대통령 재선 폭력 사태 속 치러진 선거서 과반...,20210105.0
2,"당국, 동부구치소 사망자 늑장통보 논란에 ""화장 전에 통보했다""","당국, 동부구치소 사망자 늑장통보 논란에 ""화장 전에 통보했다"" 서울동부구치소에서 ...",20210105.0
3,삼성전자 사흘째 최고가 경신…시총 500조 시대,삼성전자 사흘째 최고가 경신…시총 500조 시대 삼성전자가 5일 사흘째 최고가를 경...,20210105.0
4,"지아이그룹-세브란스병원, 혁신 신약 개발 공동연구 협약","지아이그룹세브란스병원, 혁신 신약 개발 공동연구 협약 신약 개발기업 지아이이노베이션...",20210105.0
...,...,...,...
691660,미배포 기사,<p>허리아파</p>,20220811.0
691661,dd,<p>d</p>,20220914.0
691662,ㅅㄷ,<p>ㅅㄷ</p>,20220921.0
691663,test,<p>test</p>,20220921.0


## 1차분석

In [3]:
df = pd.read_csv("fire_news.csv",encoding='utf-8-sig')

/tmp/ipykernel_26760/3281381191.py:1: DtypeWarning: Columns (42,45,46,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("fire_news.csv",encoding='utf-8-sig')


In [5]:
df.columns

Index(['_index', '_type', '_id', '_score', 'YNewsML.Header.SendTime',
       'YNewsML.Header.Action', 'YNewsML.Header.ServiceType',
       'YNewsML.Header.ContentID', 'YNewsML.Header.SendDate',
       'YNewsML.Metadata.Category.code', 'YNewsML.Metadata.Category.name',
       'YNewsML.Metadata.Credit', 'YNewsML.Metadata.DeliCode',
       'YNewsML.Metadata.Attribute.AttributeCode.code',
       'YNewsML.Metadata.Attribute.AttributeCode.name',
       'YNewsML.Metadata.Region.RegionCode.code',
       'YNewsML.Metadata.Region.RegionCode.name', 'YNewsML.Metadata.Href',
       'YNewsML.Metadata.Desk.code', 'YNewsML.Metadata.Desk.content',
       'YNewsML.Metadata.Class.ClassCode', 'YNewsML.Metadata.Urgency',
       'YNewsML.Metadata.Source', 'YNewsML.Metadata.Writer.uid',
       'YNewsML.Metadata.Writer.content', 'YNewsML.NewsContent.LangType',
       'YNewsML.NewsContent.Title', 'YNewsML.NewsContent.Body',
       'YNewsML.NewsContent.SubTitle', 'send_timestamp',
       'YNewsML.Metadata.Class

In [8]:
df1 = df[['YNewsML.NewsContent.Title','YNewsML.NewsContent.Body','NamedEntity']]

In [10]:
a = []

for i in df1['YNewsML.NewsContent.Title']:
    if '이태원' in i:
        a.append(i)

In [15]:
cond1 = df1['YNewsML.NewsContent.Title'].isin(a)

In [20]:
df1_NER = df1.loc[cond1]['NamedEntity'].dropna()

In [33]:
df1_NER = pd.DataFrame(df1_NER).reset_index().drop("index",axis=1)

In [41]:
df1_NER['NamedEntity'][0]

"[{'code': 'OG', 'word': '경찰'}, {'code': 'LC', 'word': '한강공원'}, {'code': 'LC', 'word': '이태원'}, {'code': 'AF', 'word': '자전거'}, {'code': 'OG', 'word': '서울경찰청'}, {'code': 'DT', 'word': '22일부터 다음 달 21일까지'}, {'code': 'DT', 'word': '21일'}, {'code': 'LC', 'word': '홍대'}, {'code': 'LC', 'word': '압구정'}, {'code': 'AF', 'word': '공유자전거'}, {'code': 'QT', 'word': '두 바퀴'}, {'code': 'DT', 'word': '올해'}, {'code': 'LC', 'word': '서울'}, {'code': 'QT', 'word': '14명'}, {'code': 'DT', 'word': '전날'}, {'code': 'DT', 'word': '지난해'}, {'code': 'QT', 'word': '2명'}, {'code': 'DT', 'word': '여름'}, {'code': 'DT', 'word': '3년간'}, {'code': 'DT', 'word': '2019∼2021년'}, {'code': 'DT', 'word': '7월 22일∼8월 21일'}, {'code': 'QT', 'word': '176건'}, {'code': 'DT', 'word': '30일간'}, {'code': 'DT', 'word': '6월 21일∼7월 21일'}, {'code': 'QT', 'word': '169건'}, {'code': 'QT', 'word': '4.2%'}, {'code': 'QT', 'word': '한 번'}]"